# Kedro Example

This is an example in the notebook, Kedro expect you to use it with them folders structure

In [1]:
from kedro.extras.datasets.pandas import CSVDataSet
from kedro.io import DataCatalog, MemoryDataSet
from kedro.pipeline import node, Pipeline
from kedro.runner import SequentialRunner

from darts import models

from soam.forecaster import Forecaster
from soam.savers import CSVSaver

/home/fabian/anaconda3/lib/python3.7/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [42]:
!wget https://raw.githubusercontent.com/facebook/prophet/master/examples/example_retail_sales.csv

--2020-08-19 13:26:34--  https://raw.githubusercontent.com/facebook/prophet/master/examples/example_retail_sales.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)...151.101.216.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.216.133|:443...connected.
HTTP request sent, awaiting response...200 OK
Length: 5279 (5,2K) [text/plain]
Saving to: ‘example_retail_sales.csv’

example_retail_sale 100%[===================>]   5,16K  --.-KB/s    in 0,002s  

2020-08-19 13:26:35 (3,28 MB/s) - ‘example_retail_sales.csv’ saved [5279/5279]



In [12]:
data_catalog = DataCatalog(
    {
        "retail": CSVDataSet(filepath="./example_retail_sales.csv"), 
    }
)
retail_df = data_catalog.load("retail")
retail_df.tail()

,ds,y
288,2016-01-01,400928
289,2016-02-01,413554
290,2016-03-01,460093
291,2016-04-01,450935
292,2016-05-01,471421


In [52]:
data_catalog = DataCatalog(
    {
        "retail": CSVDataSet(filepath="./example_retail_sales.csv"), 
        "predictions": CSVDataSet(filepath="./preds.csv")
    }
)

def create_forecaster():
    my_model = models.Prophet(weekly_seasonality=False, daily_seasonality=False)
    forecaster = Forecaster(my_model)
    return forecaster

create_forecaster_node = node(create_forecaster, inputs=None, outputs="forecaster")

def run_forecaster(forecaster):
    return forecaster.run(raw_series=data_catalog.load("retail"), output_length=7)

run_forecaster_node = node(run_forecaster, inputs="forecaster", outputs="preds")

def save_preds(prediction):
    data_catalog.save("predictions", prediction)
    

save_preds_node = node(save_preds, inputs="preds", outputs=None)

# Assemble nodes into a pipeline
pipeline = Pipeline([create_forecaster_node, run_forecaster_node, save_preds_node])

# Create a runner
runner = SequentialRunner()

# Run the pipeline
runner.run(pipeline, data_catalog)

INFO:kedro.pipeline.node:Running node: create_forecaster(None) -> [forecaster]
INFO:kedro.io.data_catalog:Saving data to `forecaster` (MemoryDataSet)...
INFO:kedro.runner.sequential_runner:Completed 1 out of 3 tasks
INFO:kedro.io.data_catalog:Loading data from `forecaster` (MemoryDataSet)...
INFO:kedro.pipeline.node:Running node: run_forecaster([forecaster]) -> [preds]
INFO:kedro.io.data_catalog:Loading data from `retail` (CSVDataSet)...
INFO:kedro.io.data_catalog:Saving data to `preds` (MemoryDataSet)...
INFO:kedro.runner.sequential_runner:Completed 2 out of 3 tasks
INFO:kedro.io.data_catalog:Loading data from `preds` (MemoryDataSet)...
INFO:kedro.pipeline.node:Running node: save_preds([preds]) -> None
INFO:kedro.io.data_catalog:Saving data to `predictions` (CSVDataSet)...
INFO:kedro.runner.sequential_runner:Completed 3 out of 3 tasks
INFO:kedro.runner.sequential_runner:Pipeline execution completed successfully.
  forecast_date           yhat
0    2016-06-01  466987.624670
1    2016-0

{}